In [138]:
import math


def convert_to_meters(foot,inches):
    m = (foot * 12 * 2.54 + inches * 2.54) * 10e-3
    return m

def convert_to_centimeters(foot,inches):
    cm = (foot * 12 * 2.54 + inches * 2.54)
    return cm

# This is the exact same code as "Vault 1 Maze Verification"

However, the only difference is that we will be adding 8'-8" to d2 to calculate the dose 8'-8" away from the door. 

This distance is straight beyond the plane of the door. If this point is safe, then we can confidently say the point 0.3 m beyond the intersection will also be safe since this would be a diagonal distance from the plane of the door (longer distance).

d_L will also be different

In [139]:
added_distance = convert_to_meters(8,8.04742)

In [140]:
def B1(thickness, TVL): # Calculates B factor using only 1 TVL. Important for PATIENT SCATTER
    '''
    param thickness: thickness of barrier (cm)
    param TVL: TVL (cm) where it is ALWAYS dependent on energy. Can also depend on the angle too
    '''
    B = 10**-(thickness/TVL)
    return B

def B2(thickness, TVL1, TVLe): # Calculates B factor using 2 TVLs. Important for LEAKAGE
    '''
    param thickness: thickness of barrier (cm)
    param TVL1: first TVL (cm)
    param TVLe: equilibrium TVL (cm)
    '''
    B = 10**-(1+((thickness-TVL1)/TVLe))
    return B

def H_ps_unshielded(W, d_secondary, d_scatter, a, F): # Calculates unshielded equivalent dose for PATIENT SCATTER
    '''
    param W: workload in a typical week (Gy/wk)
    param d_secondary: distance from isocenter to the point of interest (cm). Includes the 0.3 m
    param d_scatter: distance from target to isocenter. Usually 1 m
    param a: scatter fraction dependent on angle (deg) and beam energy (MV)
    param F: field size in cm^2
    '''
    H_ps_uns = (W * a * F) / (d_secondary**2 * d_scatter**2 * 400) * 10**3
    return H_ps_uns # (mSv/wk)

def H_leak_unshielded(W_L, d_leakage): # Calculates UNSHIELDED equivalent dose for LEAKAGE
    '''
    param W_L: leakage workload (Gy/wk)
    param d_leakage: distance from source of leakage to point of protection (cm)
    '''
    H_leak_uns = ((W_L*10**-3) / d_leakage**2) * 10**3 # 10**-3 is the included barriers from the head and 10**3 converts Sv to mSv
    return H_leak_uns # (mSv/wk)

In [141]:
def verify_2barrier(P, T, thickness,d,tvl_6mv_ps,tvl_18mv_ps,a_6mv,a_18mv):
    '''
    param P: design goal (mSv/wk)
    param T: occupancy factor (unitless)
    param thickness: thickness of barrier (cm)
    param d: distance from target to 0.3m beyond distal surface (cm)
    param tvl_6mv_ps: TVL for 6 MV (cm)
    param tvl_18mv_ps: TVL for 18 MV (cm)
    param a_6mv: scatter fraction for 6 MV 
    param a_18mv: scatter fraction for 18 MV
    '''
  
    H_l_6mv = H_leak_unshielded(1990, d) * B2(thickness, 34, 29)
    H_l_18mv = H_leak_unshielded(250, d) * B2(thickness, 36, 34)

    H_ps_6mv = H_ps_unshielded(520, d, 1, a_6mv, 1600) * B1(thickness, tvl_6mv_ps)
    H_ps_18mv = H_ps_unshielded(250, d, 1, a_18mv, 1600) * B1(thickness, tvl_18mv_ps)

    H_total = (H_ps_6mv + H_ps_18mv + H_l_6mv + H_l_18mv) * 1e3 # Convert to uSv/wk
    
    if H_total < (P /T) * 1e3: # Convert to uSv/wk
        print("PASS")
    else:
        print("FAIL")
    
    print("H_leak_6mv: ", H_l_6mv, "uSv/wk")
    print("H_leak_18mv: ", H_l_18mv, "uSv/wk")
    print("H_ps_6mv: ", H_ps_6mv, "uSv/wk")
    print("H_ps_18mv: ", H_ps_18mv, "uSv/wk")
    print("----------------------------------")
    print("Total dose: ", H_total, "uSv/wk")
    print("Dose limit: ", (P / T) * 1e3, "uSv/wk")
    print(f"Thickness used: {thickness} cm = {thickness /2.54} inches")
    print(f"distance from target to 0.3m beyond distal surface: {d} m")
    print(f"TVL_ps (6MV) = {tvl_6mv_ps} cm")
    print(f"TVL_ps (18MV) = {tvl_18mv_ps} cm")
    print(f"Scatter Fraction (6MV) = {a_6mv}")
    print(f"Scatter Fraction (18MV) = {a_18mv}")
    print("--------------------------------------")
    print(f"TVL_1 (6MV) = 34 cm")
    print(f"TVL_e (6MV) = 29 cm")
    print(f"TVL_1 (18MV) = 36 cm")
    print(f"TVL_e (18MV) = 34 cm")
# Important note: thickness must be in centimeters.

def verify_2barrier_vault2(P, T, thickness,d,tvl_6mv_ps,a_6mv):
    '''
    param P: design goal (mSv/wk)
    param T: occupancy factor (unitless)
    param thickness: thickness of barrier (cm)
    param d: distance from target to 0.3m beyond distal surface (cm)
    param tvl_6mv_ps: TVL for 6 MV (cm)
    param a_6mv: scatter fraction for 6 MV 
    '''
  
    H_l_6mv = H_leak_unshielded(1435.5, d) * B2(thickness, 34, 29)

    H_ps_6mv = H_ps_unshielded(319, d, 1, a_6mv, 400) * B1(thickness, tvl_6mv_ps)

    H_total = (H_ps_6mv + H_l_6mv) * 1e3 # Convert to uSv/wk
    if H_total < (P/T) * 1e3: # Convert to uSv/wk
        print("PASS")
    else:
        print("FAIL")
    print(f"H_ps_6mv: {H_ps_6mv} uSv/wk")
    print(f"H_l_6mv: {H_l_6mv} uSv/wk")
    print("Total dose: ", H_total, "uSv/wk")
    print("Dose limit: ", (P / T) * 1e3, "uSv/wk")
    print(f"Thickness used: {thickness} cm = {thickness /2.54} inches")
    print(f"TVL (6MV) = {tvl_6mv_ps} cm")
    print(f"TVL1 (6MV) = 34 cm")
    print(f"TVLe (6MV) = 29 cm")
    print(f"Scatter Fraction (6MV) = {a_6mv}")
    print(f"Workload PS (6MV) = 1435.5 Gy/wk")
    print(f"Workload Leak = 319 Gy/wk")
    print(f"distance = {d} m")


In [142]:
def find_Hs(W, U, alpha0, A0, alphaZ, Az, d_h, d_r, d_z): 
    """
    Calculates the primary beam scattered from the primary barrier.

    Parameters:
    W (float): Workload (Gy/wk) for the worst-case Wall G.
    U (float): Usage factor for the worst-case Wall G.
    alpha0 (float): Albedo of the primary barrier, dependent on material, energy, 
                    angle of incidence, and reflection.
    A0 (float): Dependent on field size and d_h (m^2).
    alphaZ (float): Albedo of Wall Z, dependent on material, energy, 
                    angle of incidence, and reflection.
    Az (float): Area of the inner maze entrance as viewed from the center of the 
                primary barrier projected onto Wall Z (m^2).
    d_h (float): Distance from the source to the proximal surface of the primary barrier (m).
    d_r (float): Distance from the center of the primary barrier, past the edge of IMW, 
                 to point B (m).
    d_z (float): Distance from the point B to the plane of the door (m).
    """

    H_s = (W * U * alpha0 * A0 * alphaZ * Az) / (d_h * d_r * d_z)**2
    return H_s


def find_Hps(W, U, a, F, alpha1, A1, d_sca, d_sec, d_zz): 
    """

    Parameters:
    
    W : workload (Gy/wk) for worst case Wall G
    U : usage factor for worst case Wall G
    a : scatter fraction dependent on scatter angle and beam energy 
    F : field size (cm^2)
    alpha1 : albedo of A1. Dependent on material, 0.5 MeV, angle of incidence, angle of reflection is 0 deg
    A1 : area of the wall at the end of the maze visible from the door (m^2). Width of A1 x maze height
    d_sca : distance from source to iso (m)
    d_sec : distance from iso to center of A1 (m)
    d_zz : distance from center of A1 to plane of door (m)
    """

    H_ps = (W * U * a * (F/400)* alpha1 * A1) / (d_sca * d_sec * d_zz)**2
    return H_ps


def find_Hls(W, U, C, alpha_ls, A1, d_sec, d_zz):
    """
    
    Parameters:

    W : workload (Gy/wk) for worst case Wall G
    U : usage factor for worst case Wall G
    C : IMRT factor (unitless)
    alpha_ls : albedo of A1, dependent on material, Co-60
               angle of incidence, and reflection is 0 deg
    A1 : area of the wall at the end of the maze visible from the door (m^2). Width of A1 x maze height
    d_sec : distance from iso to center of A1 (m) if gantry rotates uniformly
    d_zz : distance from center of A1 to plane of door (m)
    """

    H_ls = (10**-3 *W * U * C * alpha_ls * A1) / (d_sec * d_zz)**2
    return H_ls


def find_Hlt(W, C, U, t_slant, TVL1, TVLe, d_L):
    """

    Parameters:

    W : workload (Gy/wk) for worst case Wall G
    C : IMRT factor (unitless)
    U : usage factor for worst case Wall G
    t_slant : slant thickness of IMW (cm)
    TVL1 : TVL of IMW material (cm). Dependent on incident energy
    TVLe : equilibrium TVL of IMW material (cm). Dependent on incident energy
    d_L : distance from source to center of plane of door (m)
    """

    H_lt = (10**-3 * W * C * U * 10**-(1 + (t_slant - TVL1) / TVLe)) / d_L**2
    return H_lt


In [143]:
# Constants
# maze width
maze_width = convert_to_meters(8,5.00242)
# maze height
maze_height = convert_to_meters(10,0)
# inner maze height
inner_maze_height = convert_to_meters(14,0)
inner_maze_entrance = convert_to_meters(8,0)
# dz
dz = convert_to_meters(24,11.10850)
# dzz
dzz = convert_to_meters(34,0.99985)

test1 = dz / (maze_width * maze_height)**0.5

test2 = dzz / (maze_width * maze_height)**0.5

if test1 < 6 and test1 > 2:
    print('PASS')
    print(test1)
if test2 < 6 and test2 > 2:
    print('PASS')
    print(test2)

if 1 < maze_height / maze_width and maze_height / maze_width < 2:
    print('PASS')
    print(maze_height/maze_width)

W_6MV = 520 # Gy/wk
W_18MV = 250 # Gy/wk

print(f"maze height = {maze_height} m")
print(f"maze width = {maze_width} m")
print(f"inner maze entrance = {inner_maze_entrance} m")
print(f"inner maze height = {inner_maze_height} m")
print(f"dz = {dz} m")
print(f"dzz = {dzz} m")
print(f"Workload (6MV) = {W_6MV} Gy/wk")
print(f"Workload (18MV) = {W_18MV} Gy/wk")

PASS
2.7168911686986728
PASS
3.7150668752779734
PASS
1.188090344765997
maze height = 3.048 m
maze width = 2.5654614679999996 m
inner maze entrance = 2.4384 m
inner maze height = 4.267200000000001 m
dz = 7.5973559 m
dzz = 10.38859619 m
Workload (6MV) = 520 Gy/wk
Workload (18MV) = 250 Gy/wk


In [144]:
# H_s
# Variables for Vault #1
d_r = convert_to_meters(25,1.28538)
d_h = convert_to_meters(17,9.04728) + 1
area_0 = (0.40 * d_h)**2 # m
alpha_z = 13e-3 # Albedo of Wall Z, concrete, 0.5 MeV, 0 deg incidence, 60 deg reflection
alpha_0_6MV = 4e-3 # Albedo of Area 0, concrete, 6 MV, 0 deg incidence, 60 deg reflection
alpha_0_18MV = 2.5e-3 # Albedo of Area 0, concrete, 18 MV, 0 deg incidence, 60 deg reflection

# Find A_z
width_Az = convert_to_meters(7,9.78277)
area_z = width_Az * inner_maze_height

print("H_s")
print("--------------------------")
print(f"dz = {dz} m")
print(f"dzz = {dzz} m")
print(f"d_r = {d_r} m")
print(f"d_h = {d_h} m")
print("--------------------------")
print(f"alpha_0_6MV = {alpha_0_6MV}")
print(f"alpha_0_18MV = {alpha_0_18MV}")
print(f"alpha_z = {alpha_z}")
print("--------------------------")
print(f"width_Az = {width_Az} m")
print(f"area_0 = {area_0} m^2")
print(f"area_z = {area_z} m^2")

H_s_6MV = find_Hs(W_6MV, 0.25, alpha_0_6MV, area_0, alpha_z, area_z, d_h, d_r, dz)
H_s_18MV = find_Hs(W_18MV, 0.25, alpha_0_18MV, area_0, alpha_z, area_z, d_h, d_r, dz)

print("---------------------------")
print(f"H_s_6MV = {H_s_6MV} Sv/wk")
print(f"H_s_18MV = {H_s_18MV} Sv/wk")
print("--------------------------")

H_s_total = H_s_6MV + H_s_18MV
print(f"H_s_total = {H_s_total} Sv/wk")


H_s
--------------------------
dz = 7.5973559 m
dzz = 10.38859619 m
d_r = 7.652648652000001 m
d_h = 6.4114009119999995 m
--------------------------
alpha_0_6MV = 0.004
alpha_0_18MV = 0.0025
alpha_z = 0.013
--------------------------
width_Az = 2.3820823580000003 m
area_0 = 6.57696986470311 m^2
area_z = 10.164821838057604 m^2
---------------------------
H_s_6MV = 3.2525048859604977e-06 Sv/wk
H_s_18MV = 9.773151700602458e-07 Sv/wk
--------------------------
H_s_total = 4.2298200560207435e-06 Sv/wk


In [145]:
# Workload for H_ps is the same as H_s

# H_ps
d_sec = convert_to_meters(31,6.75193)
d_sca = 1 # m
a_6MV = 1.39e-3
a_18MV = 8.64e-4
alpha_1_ps = 22e-3 # albedo of A1, concrete, 0.5 MeV, 45 deg incidence, 0 deg reflection
width_A1 = convert_to_meters(10,11.97257)
area_1 = maze_height * width_A1 # m^2

print("H_ps")
print("--------------------------")
print(f"d_zz = {dzz} m")
print(f"d_sec = {d_sec} m")
print(f"d_sca = {d_sca} m")
print("--------------------------")
print(f"a_6MV = {a_6MV}")
print(f"a_18MV = {a_18MV}")
print(f"alpha_1_ps = {alpha_1_ps}")
print("---------------------------")
print(f"width_A1 = {width_A1} m")
print(f"area_1 = {area_1} m^2")

H_ps_6MV = find_Hps(W_6MV, 0.25, 1600, a_6MV, alpha_1_ps, area_1, d_sca, d_sec, dzz)
H_ps_18MV = find_Hps(W_18MV, 0.25, 1600, a_18MV, alpha_1_ps, area_1, d_sca, d_sec, dzz)

print("---------------------------")
print(f"H_ps_6MV = {H_ps_6MV} Sv/wk")
print(f"H_ps_18MV = {H_ps_18MV} Sv/wk")
print("---------------------------")

H_ps_tot = H_ps_6MV + H_ps_18MV
print(f"H_ps_tot = {H_ps_tot} Sv/wk")


H_ps
--------------------------
d_zz = 10.38859619 m
d_sec = 9.620299022000001 m
d_sca = 1 m
--------------------------
a_6MV = 0.00139
a_18MV = 0.000864
alpha_1_ps = 0.022
---------------------------
width_A1 = 3.3521032780000004 m
area_1 = 10.217210791344002 m^2
---------------------------
H_ps_6MV = 1.6266057571359304e-05 Sv/wk
H_ps_18MV = 4.8609137180597805e-06 Sv/wk
---------------------------
H_ps_tot = 2.1126971289419086e-05 Sv/wk


In [146]:
W_L_6MV = 1990 # Gy/wk
W_L_18MV = 250 # Gy/wk

print(f"Leakage Workload (6MV) = {W_L_6MV} Gy/wk")
print(f"Leakage Workload (18MV) = {W_L_18MV} Gy/wk")
print("--------------------------")

# H_lt
d_leak = convert_to_meters(26,4.88130) # m
thickness_IMW = 85.09 # cm
t_slant = convert_to_meters(3,0.54217) * 100 # cm

# TVL for 6 MV and 18 MV
tvl1_6MV = 34 # cm
tvle_6MV = 29 # cm
tvl1_18MV = 36 # cm
tvle_18MV = 34 # cm

# B factors for 6 MV and 18 MV
B_IMW_6MV = B2(t_slant, tvl1_6MV, tvle_6MV)
B_IMW_18MV = B2(t_slant, tvl1_18MV, tvle_18MV)

print("H_lt")
print("--------------------------")
print(f"d_leak = {d_leak} m")
print(f"t_slant = {t_slant} cm")
print(f"B_IMW_6MW = {B_IMW_6MV}")
print(f"B_IMW_18MV = {B_IMW_18MV}")
print("--------------------------")

H_Lt_6MV = find_Hlt(W_L_6MV, 1, 0.25, t_slant, tvl1_6MV, tvle_6MV, d_leak)
H_Lt_18MV = find_Hlt(W_L_18MV, 1, 0.25, t_slant, tvl1_18MV, tvle_18MV, d_leak)
H_lt_tot = H_Lt_6MV + H_Lt_18MV

print(f"H_Lt_6MV = {H_Lt_6MV} Sv/wk")
print(f"H_Lt_18MV = {H_Lt_18MV} Sv/wk")
print("--------------------------")
print(f"H_lt_tot = {H_lt_tot} Sv/wk")


Leakage Workload (6MV) = 1990 Gy/wk
Leakage Workload (18MV) = 250 Gy/wk
--------------------------
H_lt
--------------------------
d_leak = 8.04878502 m
t_slant = 92.81711179999999 cm
B_IMW_6MW = 0.0009371815518234737
B_IMW_18MV = 0.0021325954411002533
--------------------------
H_Lt_6MV = 7.1970771917140454e-06 Sv/wk
H_Lt_18MV = 2.057443123996608e-06 Sv/wk
--------------------------
H_lt_tot = 9.254520315710654e-06 Sv/wk


In [147]:
# H_ls

# Because of uniform gantry rotation, d_sec is the same for H_ls as H_ps
# area 1 is the same for H_ls as H_ps

alpha_1_ls = 9e-3 # albedo of A1, concrete, Co-60, 45 deg incidence, 0 deg reflection
# This is for 6 MV AND 18 MV

H_ls_6MV = find_Hls(W_L_6MV, 0.25, 1, alpha_1_ls, area_1, d_sec, dzz)
H_ls_18MV = find_Hls(W_L_18MV, 0.25, 1, alpha_1_ls, area_1, d_sec, dzz)
H_ls_tot = H_ls_6MV + H_ls_18MV
print("H_ls")
print(f"alpha_1_ls = {alpha_1_ls}")
print("--------------------------")
print(f"H_ls_6MV = {H_ls_6MV} Sv/wk")
print(f"H_ls_18MV = {H_ls_18MV} Sv/wk")
print("--------------------------")
print(f"H_ls_tot = {H_ls_tot} Sv/wk")



H_ls
alpha_1_ls = 0.009
--------------------------
H_ls_6MV = 4.580122300633979e-06 Sv/wk
H_ls_18MV = 5.753922488233643e-07 Sv/wk
--------------------------
H_ls_tot = 5.155514549457344e-06 Sv/wk


In [148]:
H_G_6MV = 0.25 * H_s_6MV + H_ps_6MV + H_Lt_6MV + H_ls_6MV
H_G_18MV = 0.40 * H_s_18MV + H_ps_18MV + H_Lt_18MV + H_ls_18MV

H_total_6MV = (2.64 * H_G_6MV) * 1e6 # uSv/wk
H_total_18MV = (2.64 * H_G_18MV) * 1e6 # uSv/wk

print(f"H_G_6MV = {H_G_6MV} Sv/wk")
print(f"H_G_18MV = {H_G_18MV} Sv/wk")
print("--------------------------")
print(f"H_total_6MV = {H_total_6MV} uSv/wk")
print(f"H_total_18MV = {H_total_18MV} uSv/wk")
print("--------------------------")

H_G_6MV = 2.8856383285197454e-05 Sv/wk
H_G_18MV = 7.884675158903851e-06 Sv/wk
--------------------------
H_total_6MV = 76.18085187292128 uSv/wk
H_total_18MV = 20.81554241950617 uSv/wk
--------------------------


# Captured Gammas for 18 MV

In [149]:
# Use Workload for 18 MV, 250 Gy/wk

K = 6.9e-16 # Sv*m^2 
TVD = 5.4 # m at 18 MV
d0 = 1 # m
d1 = convert_to_meters(28,9.23883) # m
d2 = convert_to_meters(29,4.49813) + added_distance # m

# Find phi_A
vault_with = convert_to_meters(35,5.99883) # m
vault_length = convert_to_meters(39,10.09457) # m
vault_height = convert_to_meters(14,0) # m. We assume no attenuation from the drop ceiling 20" below the vault ceiling
S_r = 2* (vault_with * vault_height + vault_length * vault_height + vault_length * vault_with) # m^2
Q_n = 0.96e12 # neutrons/Gy # For Varian 2100C
beta = 1 # for Pb head shielding

phi_A = (beta * Q_n) / (4*math.pi * d1**2) + (5.4 * beta * Q_n) / (2*math.pi * S_r) + (1.3*Q_n) / (2*math.pi*S_r)

print(f"d1 = {d1} m")
print(f"d2 = {d2} m")
print(f"S_r = {S_r} m^2")
print('--------------------------')
print(f"phi_A = {phi_A} neutrons/Gy*m^2")

H_cg = (250 * K * phi_A * 10**(-d2/TVD)) * 1e6 # uSv/wk
print(f"H_cg = {H_cg} uSv/wk")



d1 = 8.769066282 m
d2 = 11.59625697 m
S_r = 458.78026779163326 m^2
--------------------------
phi_A = 3224787620.952975 neutrons/Gy*m^2
H_cg = 3.9612770969291464 uSv/wk


# Neutron Contribution for 18 MV

In [150]:
s_0 = inner_maze_height * inner_maze_entrance
s_1 = maze_height * maze_width
TVD_n = 2.06 * math.sqrt(s_1)
print(f"s_0 = {s_0} m^2")
print(f"s_1 = {s_1} m^2")
print('--------------------------')
H_n = 2.4e-15 * 250 * phi_A * math.sqrt(s_0/s_1) * (1.64*10**(-d2/1.9) + 10**(-d2/TVD_n)) * 1e6 # uSv/wk
print(f"H_n = {H_n} uSv/wk")

s_0 = 10.405140480000002 m^2
s_1 = 7.819526554463999 m^2
--------------------------
H_n = 21.660426398189596 uSv/wk


# Door Thickness Calculations

In [151]:
H_final_uns = H_total_6MV + H_total_18MV + H_cg + H_n
print(f"H_total_6MV = {H_total_6MV} uSv/wk")
print(f"H_total_18MV = {H_total_18MV} uSv/wk")
print(f"H_cg = {H_cg} uSv/wk")
print(f"H_n = {H_n} uSv/wk")
print('--------------------------')
print(f"H_final_uns = {H_final_uns} uSv/wk")

H_total_6MV = 76.18085187292128 uSv/wk
H_total_18MV = 20.81554241950617 uSv/wk
H_cg = 3.9612770969291464 uSv/wk
H_n = 21.660426398189596 uSv/wk
--------------------------
H_final_uns = 122.6180977875462 uSv/wk


In [152]:
# To determine TVLs...

if H_Lt_6MV < 0.5 * H_G_6MV:
    print('PASS')
else:
    print('FAIL')

if H_Lt_18MV < 0.5 * H_G_18MV:
    print('PASS')
else:
    print('FAIL')

PASS
PASS


Since conditions pass for both energies, we will use 0.2 MeV for TVLs

In [153]:
tvl_pb_200kev = 0.6 # cm , used for x-rays
tvl_steel_200kev = 3 # cm , used for x-rays

tvl_bpe_100kev = 4.5 # cm , used for neutrons

tvl_steel_3600kev = 9 # cm , used for captured gammas
tvl_pb_3600kev = 6.1 # cm , used for captured gammas


We will design a door with a steel shell (two 1/16" layers of steel) and T=1/8

In [154]:
T_for_maze = 1/8 # Occupancy factor for plane of door
print(f"Occupancy factor at plane of the door = {T_for_maze}")
print('----------------------------------------')
t_steel = (T_for_maze) * 2.54 # cm
B_steel_cg = B1(t_steel, tvl_steel_3600kev)
H_cg_new = H_cg * B_steel_cg # uSv/wk
print(f"H_cg with steel = {H_cg_new} uSv/wk")

B_steel_xray = B1(t_steel, tvl_steel_200kev)
H_xray_6MV_new = H_total_6MV * B_steel_xray
H_xray_18MV_new = H_total_18MV * B_steel_xray
print('---------------------------------')
print(f"H_xray_6MV with steel = {H_xray_6MV_new} uSv/wk")
print(f"H_xray_18MV with steel = {H_xray_18MV_new} uSv/wk")

H_xray_new = H_xray_6MV_new + H_xray_18MV_new
print('---------------------------------')
print(f"H_xray with steel = {H_xray_new} uSv/wk")

H_final_with_steel = H_xray_6MV_new + H_xray_18MV_new + H_cg_new + H_n

print('---------------------------------')
print(f"H_total_with_steel (Only includes xrays and cgs) = {H_xray_18MV_new+H_xray_6MV_new+H_cg_new}")
print(f"H_neutrons = {H_n} uSv/wk")
print('------------------------------------')
print(f"H_total_with_steel (Includes xrays, neutrons and cgs) = {H_final_with_steel} uSv/wk")

Occupancy factor at plane of the door = 0.125
----------------------------------------
H_cg with steel = 3.6522243293149104 uSv/wk
---------------------------------
H_xray_6MV with steel = 59.705245907313916 uSv/wk
H_xray_18MV with steel = 16.313772402071233 uSv/wk
---------------------------------
H_xray with steel = 76.01901830938515 uSv/wk
---------------------------------
H_total_with_steel (Only includes xrays and cgs) = 79.67124263870005
H_neutrons = 21.660426398189596 uSv/wk
------------------------------------
H_total_with_steel (Includes xrays, neutrons and cgs) = 101.33166903688965 uSv/wk


In [155]:
P_door = 100 # uSv/wk
# If we use T=1/8 no shielding is required. If we want to be more conservative with T=1/5, we still dont need any lead shielding.
print(f"CG and xray contributions with steel: {H_cg_new + H_xray_6MV_new + H_xray_18MV_new} uSv/wk")

fraction = .99 # Test fraction

# Captured Gammas and Xrays
B_x_cg = (fraction * P_door) / ((H_cg_new + H_xray_6MV_new + H_xray_18MV_new) * T_for_maze)
print(f"B_x_cg = {B_x_cg}")
thickness_lead = -math.log10(B_x_cg) * tvl_pb_3600kev # cm #### WHICH TVL DO I USE ???
print(f"thickness_lead = {thickness_lead} cm = {thickness_lead/2.54} in")
      
print('---------------------------------')

# Neutrons
B_n = (1-fraction) * P_door / (H_n * T_for_maze)
print(f"B_n = {B_n}")
thickness_bpe = -math.log10(B_n) * tvl_bpe_100kev # cm
print(f"thickness_bpe = {thickness_bpe} cm = {thickness_bpe/2.54} in")


CG and xray contributions with steel: 79.67124263870006 uSv/wk
B_x_cg = 9.940851601770905
thickness_lead = -6.084283902937967 cm = -2.3953873633614045 in
---------------------------------
B_n = 0.3693371429044747
thickness_bpe = 1.9465965662532292 cm = 0.7663766008870981 in


Because B_x_cg > 1 and 87.366 < P/T = 800 uSv/wk, we technically don't need any lead shielding.
Quick note: A door that takes cares of captured gammas should be able to handle x-rays since the TVL for xrays (6mm) is much less than TVL for cgs (6.1cm).

Therefore, we use the TVL for cgs (6.1cm) to determine the thickness of lead.

# Quick verification of the thicknesses

In [156]:
# Lead thickness (cm) - might be negative
t_pb_indesign = 0.079375 # cm NO LEAD IS ACTUALLY NEEDED
t_bpe_indesign = 3.81 # cm = 1.5" of BPE

print('--------------------------')
print(f"H_final_uns = {H_final_uns} uSv/wk")
print('--------------------------')

# Neutron calculation
B_bpe = B1(t_bpe_indesign, tvl_bpe_100kev)
H_neutrons_new = H_n * B_bpe
print(f"H_neutrons_new = {H_neutrons_new} uSv/wk")

# CG calculation
B_steel_testing_cg = B1(t_steel, tvl_steel_3600kev)
if t_pb_indesign >= 0:
    B_pb_testing_cg = B1(t_pb_indesign, tvl_pb_3600kev)
    B_pb_cg = B_pb_testing_cg
else:
    B_pb_cg = 1  # No lead effect if thickness is negative

H_cg_testing_new = H_cg * B_steel_testing_cg * B_pb_cg
print(f"H_cg_testing_new = {H_cg_testing_new} uSv/wk")

# X-ray calculation
B_steel_testing_xray = B1(t_steel, tvl_steel_200kev)
if t_pb_indesign >= 0:
    B_pb_testing_xray = B1(t_pb_indesign, tvl_pb_200kev)
    B_pb_xray = B_pb_testing_xray
else:
    B_pb_xray = 1  # No lead effect if thickness is negative

H_xray_testing_new = (H_total_6MV + H_total_18MV) * B_steel_testing_xray * B_pb_xray
print(f"H_xray_testing_new = {H_xray_testing_new} uSv/wk")

# Final evaluation

H_final_shielded = H_cg_testing_new + H_xray_testing_new + H_neutrons_new
print('--------------------------')
if H_final_shielded < P_door / T_for_maze:
    print('PASS')
else:
    print('FAIL')
print(f"H_final_shielded = {H_final_shielded} uSv/wk < {P_door / T_for_maze} uSv/wk")


--------------------------
H_final_uns = 122.6180977875462 uSv/wk
--------------------------
H_neutrons_new = 3.0831903288589553 uSv/wk
H_cg_testing_new = 3.5444197685979573 uSv/wk
H_xray_testing_new = 56.05709530189087 uSv/wk
--------------------------
PASS
H_final_shielded = 62.68470539934778 uSv/wk < 800.0 uSv/wk


# Patient scatter from both vaults

In [157]:
d_from_iso_point1 = convert_to_meters(30,9.20062) # m
slanted_thickness_vault1 = convert_to_centimeters(4,1.11708) # cm
print(f"Distance from iso to point 1 in Vault 1 = {d_from_iso_point1} m")
print(f"Slanted thickness of IMW in Vault 1 = {slanted_thickness_vault1} cm")

tvl_6mv_30deg_ps = 26 # cm
tvl_18mv_30deg_ps = 32 # cm
tvl1_6mv_leakage = 34 # cm
tvle_6mv_leakage = 29 # cm
tvl1_18mv_leakage = 36 # cm
tvle_18mv_leakage = 34 # cm

a_6MV_30deg = 2.77e-3
a_18MV_30deg = 2.53e-3

print(f"TVL (6MV) at 30 deg = {tvl_6mv_30deg_ps} cm")
print(f"TVL (18MV) at 30 deg = {tvl_18mv_30deg_ps} cm")
print(f"TVL1 (6MV) = {tvl1_6mv_leakage} cm")
print(f"TVLe (6MV) = {tvle_6mv_leakage} cm")
print(f"TVL1 (18MV) = {tvl1_18mv_leakage} cm")
print(f"TVLe (18MV) = {tvle_18mv_leakage} cm")
print(f"Scatter Fraction (6MV) = {a_6MV_30deg}")
print(f"Scatter Fraction (18MV) = {a_18MV_30deg}")

Distance from iso to point 1 in Vault 1 = 9.377695748 m
Slanted thickness of IMW in Vault 1 = 124.7573832 cm
TVL (6MV) at 30 deg = 26 cm
TVL (18MV) at 30 deg = 32 cm
TVL1 (6MV) = 34 cm
TVLe (6MV) = 29 cm
TVL1 (18MV) = 36 cm
TVLe (18MV) = 34 cm
Scatter Fraction (6MV) = 0.00277
Scatter Fraction (18MV) = 0.00253


In [158]:
verify_2barrier(0.02,1/5, slanted_thickness_vault1, d_from_iso_point1, tvl_6mv_30deg_ps, tvl_18mv_30deg_ps, a_6MV_30deg,a_18MV_30deg)

PASS
H_leak_6mv:  0.001679174621758755 uSv/wk
H_leak_18mv:  0.0006970059115399828 uSv/wk
H_ps_6mv:  0.0010422932737760983 uSv/wk
H_ps_18mv:  0.0036329550095610947 uSv/wk
----------------------------------
Total dose:  7.051428816635931 uSv/wk
Dose limit:  99.99999999999999 uSv/wk
Thickness used: 124.7573832 cm = 49.11708 inches
distance from target to 0.3m beyond distal surface: 9.377695748 m
TVL_ps (6MV) = 26 cm
TVL_ps (18MV) = 32 cm
Scatter Fraction (6MV) = 0.00277
Scatter Fraction (18MV) = 0.00253
--------------------------------------
TVL_1 (6MV) = 34 cm
TVL_e (6MV) = 29 cm
TVL_1 (18MV) = 36 cm
TVL_e (18MV) = 34 cm


In [159]:
slanted_thickness_vault2 = convert_to_centimeters(3,7.17006) # cm
d_from_iso_point1_vault2 = convert_to_meters(30,8.96553) # m
print(f"Slanted thickness of IMW in Vault 2 = {slanted_thickness_vault2} cm")
print(f"Distance from iso to Point 1 in Vault 2 = {d_from_iso_point1_vault2} m")

verify_2barrier_vault2(0.02,1/5,slanted_thickness_vault2,d_from_iso_point1_vault2,tvl_6mv_30deg_ps,a_6MV_30deg)

Slanted thickness of IMW in Vault 2 = 109.6519524 cm
Distance from iso to Point 1 in Vault 2 = 9.371724462 m
PASS
H_ps_6mv: 0.0006098859725651321 uSv/wk
H_l_6mv: 0.004024168527638475 uSv/wk
Total dose:  4.634054500203607 uSv/wk
Dose limit:  99.99999999999999 uSv/wk
Thickness used: 109.6519524 cm = 43.17006 inches
TVL (6MV) = 26 cm
TVL1 (6MV) = 34 cm
TVLe (6MV) = 29 cm
Scatter Fraction (6MV) = 0.00277
Workload PS (6MV) = 1435.5 Gy/wk
Workload Leak = 319 Gy/wk
distance = 9.371724462 m


In [160]:
limit = 100 - 4.634054 - 7.0514288
print(f"Maze contributions cannot exceed {limit} uSv/wk")

Maze contributions cannot exceed 88.3145172 uSv/wk
